In [ ]:
!pip install mlxtend
!pip install ucimlrepo

In [2]:
import os
import pickle
import numpy as np
import skimage.io
import copy
import random
import sklearn
import sklearn.metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from skimage import filters
import pandas as pd
import warnings
import pickle
from scipy.stats import kendalltau
from sklearn.feature_selection import RFE
from sklearn.inspection import permutation_importance

from matplotlib import pyplot as plt
import time
from sklearn.utils import resample
from scipy.stats import norm, gaussian_kde
from sklearn.neighbors import KernelDensity
import csv
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn import datasets
import seaborn as sns
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.datasets import fetch_california_housing
from sklearn.datasets import load_wine
import itertools
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import VarianceThreshold

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.inspection import permutation_importance


def remove_highly_correlated_features_pd(X, threshold):
    # Convert X to a pandas DataFrame
    # Calculate the correlation matrix
    df = pd.DataFrame(X)
    corr_matrix = df.corr().abs()
    # Create a mask to remove the upper triangle of the correlation matrix
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

    # Set the upper triangle values to NaN
    corr_matrix.mask(mask, inplace=True)

    # Find the highly correlated features
    cols_to_drop = [column for column in corr_matrix.columns if any(corr_matrix[column] > threshold)]

    # Drop the highly correlated features from the DataFrame
    reduced_df = df.drop(columns=cols_to_drop)

    # Convert the reduced DataFrame back to numpy array
    X_reduced = reduced_df.to_numpy()

    return X_reduced, cols_to_drop


def calculate_entropy(data):
    if np.var(data) == 0:
        return 0

    scipy_kernel = gaussian_kde(data)

    #  We calculate the bandwidth for later use
    optimal_bandwidth = scipy_kernel.factor * np.std(data)

    # Calculate KDE for the entire dataset
    kde = gaussian_kde(data, bw_method=optimal_bandwidth)

    # Create a range of values to represent the KDE
    x = np.linspace(np.min(data), np.max(data), 1000)

    # Evaluate the density at each point in the range
    density = kde(x)

    # Normalize the density function
    normalized_density = density / np.sum(density * (x[1] - x[0]))

    # Calculate the probabilities of positive and negative values
    positive_probability = np.sum(normalized_density[x >= 0] * (x[1] - x[0]))
    negative_probability = np.sum(normalized_density[x < 0] * (x[1] - x[0]))

    if positive_probability == 0 or negative_probability == 0:
        sign_entropy = 0
    else:
        sign_entropy = -positive_probability * np.log2(positive_probability) \
                       - negative_probability * np.log2(negative_probability)

    return sign_entropy


def calculate_entropies(result_matrix):
    sign_entropies = []
    for column in range(result_matrix.shape[1]):
        data = result_matrix[:, column]
        sign_entropy = calculate_entropy(data)
        sign_entropies.append(sign_entropy)

    sign_entropies = np.array(sign_entropies)

    return sign_entropies


def get_unstable_features(X, y, model, bs_indices, num_bootstraps=None):
    coeffs_bs = []
    if num_bootstraps != None:
      print("bootstraps generated")
      bs_indices = []
      for i in np.arange(0, num_bootstraps, step=1):
        max_bs_range = X.shape[0]
        indx_bs = random.choices(range(max_bs_range), k=max_bs_range)
        bs_indices.append(indx_bs)

    for i in range(len(bs_indices)):
      indices = bs_indices[i]
      X_sample, y_sample = X[indices], y[indices]
      model.fit(X_sample, y_sample)
      coeffs_bs.append(model.coef_)
    coeffs_bs = np.array(coeffs_bs)

    sign_entropies = []
    for column in range(coeffs_bs.shape[1]):
        data = coeffs_bs[:, column]
        sign_entropy = calculate_entropy(data)
        sign_entropies.append(sign_entropy)

    num_predictors = len(sign_entropies)
    sign_entropies = np.array(sign_entropies)
    av_sign_entropy = np.mean(sign_entropies)
    ratio_zero_entropy = np.count_nonzero(sign_entropies == 0) / (sign_entropies.shape[0])

    non_zero_indices = np.where(sign_entropies != 0)[0]
    zero_ent_indices = np.where(sign_entropies == 0)[0]



    return non_zero_indices, zero_ent_indices, sign_entropies, coeffs_bs


def evaluate_feature_selection_method(feature_selector, X_train, y_train, X_test, y_test,eval_indices_bs,
                                      selector_params={}, model_params={}):
    """
    Evaluate a given feature selection method on training and test data.

    Parameters:
    - feature_selector: function, a feature selection method that returns selected feature indices.
    - X_train, y_train, X_test, y_test: train and test datasets.
    - selector_params: dict, parameters for the feature selector.
    - model_params: dict, parameters for the model.
    - i: iteration index, useful for storing results in dictionaries.

    Returns:
    - rmse: Root Mean Squared Error using the selected features.
    - sign_entropies: Sign entropies of the selected features.
    """

    # Apply feature selection
    selected_indices = feature_selector(X_train, y_train, **selector_params)
    # print("Selected Features: ", selected_indices)

    # Subset data using selected features
    X_train_selected = X_train[:, selected_indices]
    X_test_selected = X_test[:, selected_indices]

    # Fit and predict using the model
    model = selector_params['estimator']
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    evaluation_model = selector_params['evaluator']

    _, _, sign_entropies, coeffs_bs = get_unstable_features(X_test_selected, y_test, evaluation_model, bs_indices=eval_indices_bs)

    return selected_indices, rmse, sign_entropies, coeffs_bs


def sefe_select_features(X_train, y_train, num_bootstrap=1000, slack=0.0, **kwargs):
    model = kwargs['estimator']
    num_iter = 10
    tolerance_limit = 1
    tolerance_cur=0
    final_coeffs = np.zeros((num_bootstrap, X_train.shape[1]))

    train_mat_sel_idx = np.zeros(X_train.shape[1])
    for iter in range(num_iter):
        zero_indices = np.where(train_mat_sel_idx == 0)[0] # get only those indices which had zero entropy from previous run

        if len(zero_indices)==0:
            print("No feature of zero entropy")
            break

        coeffs_bs = []
        for i in range(num_bootstrap):
            indices_bs = random.choices(range(X_train.shape[0]), k=X_train.shape[0])
            X_sample, y_sample = X_train[indices_bs][:, zero_indices], y_train[indices_bs]
            model.fit(X_sample, y_sample)
            coeffs_bs.append(model.coef_)

        coeffs_bs = np.array(coeffs_bs)

        sign_entropies = []
        for column in range(coeffs_bs.shape[1]):
            data = coeffs_bs[:, column]
            sign_entropy = calculate_entropy(data)
            sign_entropies.append(sign_entropy)

        sign_entropies = np.array(sign_entropies)

        non_zero_indices = np.where(sign_entropies != 0)[0]
        zero_ent_indices = np.where(sign_entropies == 0)[0]

        #non_zero_indices = np.where(sign_entropies > (0+slack))[0]
        #zero_ent_indices = np.where(sign_entropies <= (0+slack))[0]

        original_0_indices = np.where(train_mat_sel_idx == 0)[0]
        mapped_non0_indices = original_0_indices[non_zero_indices]

        if not np.size(mapped_non0_indices) == 0:
            train_mat_sel_idx[mapped_non0_indices] = 1
            tolerance_cur = 0
        else:
            if tolerance_cur == 0:
                final_coeffs = coeffs_bs
                tolerance_cur = tolerance_cur + 1
            else:
                if tolerance_cur < tolerance_limit:
                    tolerance_cur = tolerance_cur + 1  # re-evaluating n times after a good run
                else:
                    break  # terminate the feature elimination process

    top_idx = np.array(np.where(train_mat_sel_idx==0)[0])
    if (len(top_idx)) == 0:
        top_idx = np.arange(0, len(train_mat_sel_idx), step=1)

    return top_idx


def boruta_select_features_old(X_train, y_train, **kwargs):
    k = kwargs.get('n_features_to_select', X_train.shape[1])  # default to all features if not provided
    model = kwargs['estimator']
    feat_selector = BorutaPy(model, n_estimators='auto', verbose=0, random_state=42)
    feat_selector.fit(X_train, y_train)
    # Get ranking of the features
    ranking = feat_selector.ranking_
    # Sort features by ranking
    top_k_idx = np.argsort(ranking)[:k]
    return top_k_idx



def rfe_select_features(X_train, y_train, **kwargs):
    estimator = kwargs['estimator']
    max_k = X_train.shape[1]  # Maximum number of features

    param_grid = {'n_features_to_select': range(1, max_k + 1)}  # Define range of k values
    selector = RFE(estimator=estimator)
    grid_search = GridSearchCV(selector, param_grid, cv=5)  # 5-fold cross-validation
    grid_search.fit(X_train, y_train)  # X_train and y_train are your training data

    best_k = grid_search.best_params_['n_features_to_select']
    selected_features = [i for i, selected in enumerate(grid_search.best_estimator_.support_) if selected]

    return selected_features


def forward_select_features(X_train, y_train, **kwargs):
    estimator = kwargs['estimator']
    n_features_to_select = kwargs.get('n_features_to_select', X_train.shape[1])

    selector = SequentialFeatureSelector(estimator=estimator, k_features=(1, X_train.shape[1]), cv=5, forward=True)
    selector.fit(X_train, y_train)
    return selector.k_feature_idx_


def sbs_select_features(X_train, y_train, **kwargs):
    estimator = kwargs['estimator']
    n_features_to_select = kwargs.get('n_features_to_select', X_train.shape[1])
    selector = SequentialFeatureSelector(estimator=estimator, k_features=(1, X_train.shape[1]), cv=5, forward=False)
    selector.fit(X_train, y_train)
    return selector.k_feature_idx_


def bidirectional_select_features(X_train, y_train, **kwargs):
    estimator = kwargs['estimator']
    n_features_to_select = kwargs.get('n_features_to_select', X_train.shape[1])
    selector = SequentialFeatureSelector(estimator=estimator, k_features=(1, X_train.shape[1]), cv=5, forward=True,
                                         floating=True)
    selector.fit(X_train, y_train)
    return selector.k_feature_idx_


def split_data_into_parts(X, y, K):
    """
    Split the feature matrix X and target values y into K approximately equal-sized parts.

    Parameters:
        X (array-like): The feature matrix.
        y (array-like): The target values.
        K (int): The number of parts to split the data into.

    Returns:
        List of tuples: Each tuple contains the feature matrix and target values for one part.
    """
    num_samples = X.shape[0]
    part_size = num_samples // K
    parts = []

    # Shuffle the data randomly
    indices = np.random.permutation(num_samples)
    X_shuffled = X[indices]
    y_shuffled = y[indices]

    # Split the shuffled data into K parts
    for i in range(K):
        start_idx = i * part_size
        end_idx = start_idx + part_size if i < K - 1 else num_samples
        X_part = X_shuffled[start_idx:end_idx]
        y_part = y_shuffled[start_idx:end_idx]
        parts.append((X_part, y_part))

    return parts

In [4]:
#### BAY COFE SPECIFIC

import numpy as np
import matplotlib.pyplot as plt
import gc
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import numpy as np
import itertools
import random
import pandas as pd
from scipy.stats import norm, gaussian_kde
from sklearn.neighbors import KernelDensity
import pickle
from sklearn.linear_model import Ridge
from scipy.stats import norm
import os


In [5]:
def calculate_entropy_dist(mu, sigma):
    #p_positive = 1 - norm.cdf(-mu / sigma)
    #p_negative = norm.cdf(-mu / sigma)
    p_negative = norm.cdf(0, mu, sigma)
    p_positive = 1 - p_negative

    # Avoiding log(0) issue by adding a small epsilon where probabilities are 0
    epsilon = 1e-10
    entropy = -(p_positive * np.log2(p_positive + epsilon) + p_negative * np.log2(p_negative + epsilon))

    return entropy

In [6]:
def baycofe_select_features(X_train, y_train, slack=0.001, **kwargs): #housing: 0.001 energy: 0.025 superconductivity: 0.005
    model = kwargs['estimator']
    num_iter = 10
    tolerance_limit = 1
    tolerance_cur=0
    #final_coeffs = np.zeros((num_bootstrap, X_train.shape[1]))

    train_mat_sel_idx = np.zeros(X_train.shape[1])
    for iter in range(num_iter):
        zero_indices = np.where(train_mat_sel_idx == 0)[0] # get only those indices which had zero entropy from previous run

        if len(zero_indices)==0:
            print("No feature of zero entropy")
            break

        X_selected = X_train[:, zero_indices]
        model.fit(X_selected, y_train)

        beta_means = model.coef_
        beta_stds = np.sqrt(np.diag(model.sigma_))

        sign_entropies = []
        for beta_mean, beta_std in zip(beta_means, beta_stds):
            sign_entropies.append(calculate_entropy_dist(beta_mean, beta_std))

        sign_entropies = np.array(sign_entropies)
        non_zero_indices = np.where(sign_entropies > slack)[0]
        zero_ent_indices = np.where(sign_entropies <= slack)[0]

        #non_zero_indices = np.where(sign_entropies > (0+slack))[0]
        #zero_ent_indices = np.where(sign_entropies <= (0+slack))[0]

        original_0_indices = np.where(train_mat_sel_idx == 0)[0]
        mapped_non0_indices = original_0_indices[non_zero_indices]

        if not np.size(mapped_non0_indices) == 0:
            train_mat_sel_idx[mapped_non0_indices] = 1
            tolerance_cur = 0
        else:
            if tolerance_cur == 0:
                #final_coeffs = coeffs_bs
                tolerance_cur = tolerance_cur + 1
            else:
                if tolerance_cur < tolerance_limit:
                    tolerance_cur = tolerance_cur + 1  # re-evaluating n times after a good run
                else:
                    break  # terminate the feature elimination process

    top_idx = np.array(np.where(train_mat_sel_idx==0)[0])
    if (len(top_idx)) == 0:
        top_idx = np.arange(0, len(train_mat_sel_idx), step=1)

    return top_idx

def remove_highly_correlated_features_pd(X, threshold):
    # Convert X to a pandas DataFrame
    # Calculate the correlation matrix
    df = pd.DataFrame(X)
    corr_matrix = df.corr().abs()
    # Create a mask to remove the upper triangle of the correlation matrix
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

    # Set the upper triangle values to NaN
    corr_matrix.mask(mask, inplace=True)

    # Find the highly correlated features
    cols_to_drop = [column for column in corr_matrix.columns if any(corr_matrix[column] > threshold)]

    # Drop the highly correlated features from the DataFrame
    reduced_df = df.drop(columns=cols_to_drop)

    # Convert the reduced DataFrame back to numpy array
    X_reduced = reduced_df.to_numpy()

    return X_reduced, cols_to_drop


def calculate_entropy(data):
    if np.var(data) == 0:
        return 0

    scipy_kernel = gaussian_kde(data)

    #  We calculate the bandwidth for later use
    optimal_bandwidth = scipy_kernel.factor * np.std(data)

    # Calculate KDE for the entire dataset
    kde = gaussian_kde(data, bw_method=optimal_bandwidth)

    # Create a range of values to represent the KDE
    x = np.linspace(np.min(data), np.max(data), 1000)

    # Evaluate the density at each point in the range
    density = kde(x)

    # Normalize the density function
    normalized_density = density / np.sum(density * (x[1] - x[0]))

    # Calculate the probabilities of positive and negative values
    positive_probability = np.sum(normalized_density[x >= 0] * (x[1] - x[0]))
    negative_probability = np.sum(normalized_density[x < 0] * (x[1] - x[0]))

    if positive_probability == 0 or negative_probability == 0:
        sign_entropy = 0
    else:
        sign_entropy = -positive_probability * np.log2(positive_probability) \
                       - negative_probability * np.log2(negative_probability)

    return sign_entropy


def calculate_entropies(result_matrix):
    sign_entropies = []
    for column in range(result_matrix.shape[1]):
        data = result_matrix[:, column]
        sign_entropy = calculate_entropy(data)
        sign_entropies.append(sign_entropy)

    sign_entropies = np.array(sign_entropies)

    return sign_entropies



def get_unstable_features(X, y, model, bs_indices, num_bootstraps=None):
    coeffs_bs = []
    if num_bootstraps != None:
      print("bootstraps generated")
      bs_indices = []
      for i in np.arange(0, num_bootstraps, step=1):
        max_bs_range = X.shape[0]
        indx_bs = random.choices(range(max_bs_range), k=max_bs_range)
        bs_indices.append(indx_bs)

    for i in range(len(bs_indices)):
      indices = bs_indices[i]
      X_sample, y_sample = X[indices], y[indices]
      model.fit(X_sample, y_sample)
      coeffs_bs.append(model.coef_)
    coeffs_bs = np.array(coeffs_bs)

    sign_entropies = []
    for column in range(coeffs_bs.shape[1]):
        data = coeffs_bs[:, column]
        sign_entropy = calculate_entropy(data)
        sign_entropies.append(sign_entropy)

    num_predictors = len(sign_entropies)
    sign_entropies = np.array(sign_entropies)
    av_sign_entropy = np.mean(sign_entropies)
    ratio_zero_entropy = np.count_nonzero(sign_entropies == 0) / (sign_entropies.shape[0])

    non_zero_indices = np.where(sign_entropies != 0)[0]
    zero_ent_indices = np.where(sign_entropies == 0)[0]

    return non_zero_indices, zero_ent_indices, sign_entropies, coeffs_bs


def evaluate_feature_selection_method(feature_selector, X_train, y_train, X_test, y_test, eval_indices_bs,
                                      selector_params={}, model_params={}):
    selected_indices = feature_selector(X_train, y_train, **selector_params)

    X_train_selected = X_train[:, selected_indices]
    X_test_selected = X_test[:, selected_indices]
    model = selector_params['estimator']
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    _, _, sign_entropies, coeffs_bs = get_unstable_features(X_test_selected, y_test, model, eval_indices_bs)
    return selected_indices, rmse, sign_entropies, coeffs_bs

def split_data_into_parts(X, y, K):
    """
    Split the feature matrix X and target values y into K approximately equal-sized parts.

    Parameters:
        X (array-like): The feature matrix.
        y (array-like): The target values.
        K (int): The number of parts to split the data into.

    Returns:
        List of tuples: Each tuple contains the feature matrix and target values for one part.
    """
    num_samples = X.shape[0]
    part_size = num_samples // K
    parts = []

    # Shuffle the data randomly
    indices = np.random.permutation(num_samples)
    X_shuffled = X[indices]
    y_shuffled = y[indices]

    # Split the shuffled data into K parts
    for i in range(K):
        start_idx = i * part_size
        end_idx = start_idx + part_size if i < K - 1 else num_samples
        X_part = X_shuffled[start_idx:end_idx]
        y_part = y_shuffled[start_idx:end_idx]
        parts.append((X_part, y_part))

    return parts

In [17]:
def get_energy_XY():
  # Load the dataset into a pandas DataFrame
  url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv"
  df = pd.read_csv(url)
  X = df.drop(columns=['Appliances', 'date', 'lights'])  # Remove 'Appliances', 'date', and 'lights' columns as features
  y = df['Appliances'].values  # Target variable is 'Appliances'

  threshold = 0.80
  X_reduced, removed_feature_names = remove_highly_correlated_features_pd(X,threshold)
  X = X_reduced
  print(removed_feature_names)

  print("Shape of X:", X.shape)
  print("Shape of y:", y.shape)
  print(X)
  # Scale features
  X_min = X.min(axis=0)
  X_max = X.max(axis=0)
  X_scaled = (X - X_min) / (X_max - X_min)
  X = X_scaled

  return X, y

In [21]:
#download the data from: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data and out train.csv on the same dir as this notebook

def get_housing_XY():
  df = pd.read_csv('train.csv', sep=',')
  df.head()
  # List of columns representing categorical variables
  categorical_columns = ['Id','MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
                        'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
                        'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
                        'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond',
                        'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                        'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional',
                        'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond',
                        'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'MoSold', 'YrSold', 'SaleType',
                        'SaleCondition']
  # Remove categorical columns from the DataFrame
  numeric_df = df.drop(columns=categorical_columns, errors='ignore')
  numeric_df = numeric_df.interpolate()
  X = numeric_df.iloc[:, :-1]  # All columns except the last one
  y = numeric_df.iloc[:, -1].values   # Last column

  threshold = 0.80
  X_reduced, removed_feature_names = remove_highly_correlated_features_pd(X,threshold)
  X = X_reduced
  print(removed_feature_names)

  print("Shape of X:", X.shape)
  print("Shape of y:", y.shape)
  print(X)
  # Scale features
  X_min = X.min(axis=0)
  X_max = X.max(axis=0)
  X_scaled = (X - X_min) / (X_max - X_min)
  X = X_scaled
  return X, y

In [22]:
def get_superconductivity_XY():
  superconductivty_data = fetch_ucirepo(id=464)

  # data (as pandas dataframes)
  X = superconductivty_data.data.features
  y = superconductivty_data.data.targets

  threshold = 0.80
  X_reduced, removed_feature_names = remove_highly_correlated_features_pd(X,threshold)
  X = X_reduced
  print(removed_feature_names)

  print("Shape of X:", X.shape)
  print("Shape of y:", y.shape)
  # Scale features
  X_min = X.min(axis=0)
  X_max = X.max(axis=0)
  X_scaled = (X - X_min) / (X_max - X_min)
  X = X_scaled
  print(X)

  return X, y

In [23]:
def get_XY(dataset_name):
  if dataset_name == 'superconductivity':
    X, y = get_superconductivity_XY()
  elif dataset_name == 'housing':
    X, y = get_housing_XY()
  elif dataset_name == 'energy':
    X, y = get_energy_XY()
  else:
    print("Wrong Dataset Name")
    X = y = None

  return X, y

In [ ]:
### ALL FEATURE SELECTION
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

k = 5
num_folds = 5
alpha_val = 0.1
num_runs = 10 #change if required
num_bootstraps = 1000 #change if required
csem_pert_range = 0.2
num_perturb = 1000
csem_iter = 10
save_dir = '/content/' # makedir to store results

model_params = {
    'alpha': 1
}


csem_params = {
    'pert_range': csem_pert_range,
    'n_perturb': num_perturb,
    'n_iter': csem_iter
}

model_params = {
    'alpha': 1
}

dataset_names = ['housing', 'superconductivity', 'energy']#['superconductivity'] #, 'housing', 'energy']

lambda_inits = [0, 0.1, 1, 5]

for dataset_name in dataset_names:
  X, y = get_XY(dataset_name)
  for lambda_init in lambda_inits:
    pkl_filename = f'{save_dir}{dataset_name}_bayCoFE_R_{str(lambda_init*10)}_numfolds_CVk_{num_folds}.pkl'

    if os.path.exists(pkl_filename):
      print(f"{pkl_filename} Exists... skipping...")
      continue

    if lambda_init== 0:
      model = BayesianRidge(lambda_init=1e-10, fit_intercept=True, compute_score=True)
    else:
      model = BayesianRidge(lambda_init=lambda_init, fit_intercept=True, compute_score=True)

    selected_features_dict = {}
    rmse_dict = {}
    entropy_dict = {}
    coeffs_dict = {}

    for i in range(num_runs):
        print(f"#############{dataset_name}####{lambda_init}############# {i}")
        data_parts = split_data_into_parts(X, y, num_folds)
        part_permutations = itertools.permutations(data_parts, 2)
        for perm in part_permutations:
            train_data, test_data = perm
            X_train, y_train = train_data
            X_test, y_test = test_data

            indices_bs = [random.choices(range(X_test.shape[0]), k=X_test.shape[0]) for _ in range(num_bootstraps)]

            # baycofe feature selection
            selected_indices_sem, rmse_sem, sign_entropies_sem, coeffs_bs_sem = \
                evaluate_feature_selection_method(baycofe_select_features, X_train, y_train, X_test, y_test, indices_bs,
                                                  selector_params={'estimator': model,
                                                                  'evaluator': model},
                                                  model_params={'sigma_threshold': 1.5, 'iter_max': 10, 'n_convergence_iters': 3})

            selected_features_dict.setdefault('baycofe', []).append(selected_indices_sem)
            rmse_dict.setdefault('baycofe', []).append(rmse_sem)
            entropy_dict.setdefault('baycofe', []).append(sign_entropies_sem)
            coeffs_dict.setdefault('baycofe', []).append(coeffs_bs_sem)
            print("Entropy BayCOFE:", np.mean(sign_entropies_sem))
            print("BayCOFE selected features:", selected_indices_sem)
            print("RMSE BayCOFE:", rmse_sem)

            ###cofe
            selected_indices_sefe, rmse_sefe, sign_entropies_sefe, coeffs_bs_sefe = \
                evaluate_feature_selection_method(sefe_select_features, X_train, y_train,
                                                  X_test, y_test,
                                                  selector_params={'estimator': model,
                                                                  'evaluator': model,
                                                                  'n_features_to_select': k},
                                                  model_params=model_params,
                                                  eval_indices_bs=indices_bs)
            k = len(selected_indices_sefe)

            selected_features_dict.setdefault('cofe', []).append(selected_indices_sefe)
            rmse_dict.setdefault('cofe', []).append(rmse_sefe)
            entropy_dict.setdefault('cofe', []).append(sign_entropies_sefe)
            coeffs_dict.setdefault('cofe', []).append(coeffs_bs_sefe)
            print("Entropy COFE:", np.mean(sign_entropies_sefe))
            print("COFE selected features:", selected_indices_sefe)
            print("rmse COFE:", rmse_sefe)

            # RFE
            selected_indices_rfe, rmse_rfe, sign_entropies_rfe, coeffs_bs_rfe = evaluate_feature_selection_method(
                rfe_select_features, X_train,
                y_train, X_test, y_test, selector_params={'estimator': model,
                                                          'evaluator': model,
                                                          'n_features_to_select': k,
                                                          'n_iter': csem_params['n_iter']},
                model_params=model_params,
                eval_indices_bs=indices_bs)
            # rmse_dict[f'rfe_{i}'] = rmse_rfe
            # entropy_dict[f'rfe_{i}'] = sign_entropies_rfe
            # selected_features_dict[f'rfe_{i}'] = selected_indices_rfe
            selected_features_dict.setdefault('rfe', []).append(selected_indices_rfe)
            rmse_dict.setdefault('rfe', []).append(rmse_rfe)
            entropy_dict.setdefault('rfe', []).append(sign_entropies_rfe)
            coeffs_dict.setdefault('rfe', []).append(coeffs_bs_rfe)
            #print("RFE num features:", selected_indices_rfe)
            print("Entropy rfe:", np.mean(sign_entropies_rfe))
            print("rmse rfe:", rmse_rfe)


            ##SFS
            selected_indices_sfs, rmse_sfs, sign_entropies_sfs, coeffs_bs_sfs = evaluate_feature_selection_method(
                forward_select_features, X_train, y_train, X_test, y_test,
                selector_params={'estimator': model, 'evaluator': model,
                                'n_features_to_select': k},
                model_params=model_params, eval_indices_bs=indices_bs)
            # selected_features_dict[f'sfs_{i}'] = selected_indices_sfs
            # rmse_dict[f'sfs_{i}'] = rmse_sfs
            # entropy_dict[f'sfs_{i}'] = sign_entropies_sfs
            selected_features_dict.setdefault('sfs', []).append(selected_indices_sfs)
            rmse_dict.setdefault('sfs', []).append(rmse_sfs)
            entropy_dict.setdefault('sfs', []).append(sign_entropies_sfs)
            coeffs_dict.setdefault('sfs', []).append(coeffs_bs_sfs)
            print("Entropy SFS:", np.mean(sign_entropies_sfs))
            print("rmse SFS:", rmse_sfs)

            # SBS
            selected_indices_sbs, rmse_sbs, sign_entropies_sbs, coeffs_bs_sbs = evaluate_feature_selection_method(
                sbs_select_features, X_train, y_train, X_test, y_test,
                selector_params={'estimator': model, 'evaluator': model,
                                'n_features_to_select': k},
                model_params=model_params, eval_indices_bs=indices_bs)
            # selected_features_dict[f'sbs_{i}'] = selected_indices_sbs
            # rmse_dict[f'sbs_{i}'] = rmse_sbs
            # entropy_dict[f'sbs_{i}'] = sign_entropies_sbs
            selected_features_dict.setdefault('sbs', []).append(selected_indices_sbs)
            rmse_dict.setdefault('sbs', []).append(rmse_sbs)
            entropy_dict.setdefault('sbs', []).append(sign_entropies_sbs)
            coeffs_dict.setdefault('sbs', []).append(coeffs_bs_sbs)
            print("Entropy SBS:", np.mean(sign_entropies_sbs))
            print("rmse SBS:", rmse_sbs)

            # Bidirectional
            selected_indices_bidirectional, rmse_bidirectional, sign_entropies_bidirectional, coeffs_bs_bidirectional = evaluate_feature_selection_method(
                bidirectional_select_features, X_train, y_train, X_test, y_test,
                selector_params={'estimator': model, 'evaluator': model,
                                'n_features_to_select': k},
                model_params=model_params, eval_indices_bs=indices_bs)
            # selected_features_dict[f'bidirectional_{i}'] = selected_indices_bidirectional
            # rmse_dict[f'bidirectional_{i}'] = rmse_bidirectional
            # entropy_dict[f'bidirectional_{i}'] = sign_entropies_bidirectional
            selected_features_dict.setdefault('bidirectional', []).append(selected_indices_bidirectional)
            rmse_dict.setdefault('bidirectional', []).append(rmse_bidirectional)
            entropy_dict.setdefault('bidirectional', []).append(sign_entropies_bidirectional)
            coeffs_dict.setdefault('bidirectional', []).append(coeffs_bs_bidirectional)
            print("Entropy Bidirectional:", np.mean(sign_entropies_bidirectional))
            print("rmse bidir:", rmse_bidirectional)

            print("##############################")

    res_dict = {'selected_features': selected_features_dict,
                'rmse': rmse_dict,
                'entropy': entropy_dict,
                'coeffs': coeffs_dict
                }


    print(pkl_filename)
    with open(pkl_filename, 'wb') as f1:
        pickle.dump(res_dict, f1)

    del selected_features_dict, rmse_dict, entropy_dict, coeffs_dict